# <font color='PURPLE'>Ciência dos Dados #NaPrática 
# <font color='BLUE'> Processamento de Linguagem Natural



**Problema de Negócio**: Precisamos Criar um sistema de Categorização de Produtos Automatizada

## Bag of Words


O modelo de "saco de palavras" é uma representação simplificada usada no processamento de linguagem natural e recuperação de informação. Neste modelo, um texto (como uma sentença ou um documento) é representado como o saco (multiset) de suas palavras, desconsiderando a gramática e até a ordem das palavras, mas mantendo a multiplicidade com as palavras.

Na classificação de documentos, um saco de palavras é um vetor esparso de ocorrência de contagens de palavras; Ou seja, um histograma esparso sobre o vocabulário.

### Carregando um Dataset de um Site de E-commerce (em português)

In [0]:
import gzip
import json

In [0]:
# Carregando o dataset
corpus = list()
with gzip.open('ecommerce.json.gz') as fp:
    for line in fp:
        entry = line.decode('utf8')
        corpus.append(json.loads(entry))

In [0]:
from pprint import pprint
pprint(corpus[0])

In [0]:
print (corpus[0]['descr'])

## Gensim - Modelagem de Tópicos

https://github.com/RaRe-Technologies/gensim

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
!pip install gensim

In [0]:
import gensim
print (gensim.summarization.summarize(corpus[2]['descr']))

In [0]:
len(corpus)

In [0]:
# Construindo um classificador para produtos e categorias (considerando apenas os 10 mil primeiros produtos)
dataset = list()
for entry in corpus[:10000]:
    if 'cat' in entry:
        dataset.append( (entry['name'], entry['cat'].lower().strip()) )

In [0]:
len(dataset)

In [0]:
pprint(dataset[:2])

In [0]:
# Quantas categorias distintas nós temos e quantos itens por categoria?
from collections import Counter
counter = Counter([cat for prod, cat in dataset])
pprint(counter.most_common())

# Construindo um Classificador SVM com Bag of Words

http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
import nltk
stopwords = nltk.download('stopwords')

In [0]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')
stopwords


In [0]:
# Construindo o modelo SVM com Pipeline
modelo = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel = 'linear', probability = True))])

In [0]:
?LabelEncoder

In [0]:
# Objeto para Normalização dos labels
encoder = LabelEncoder()

In [0]:
# Obtendo dados e labels
data = [prod for prod, cat in dataset]
labels = [cat for prod, cat in dataset]
len(data)

In [0]:
# Normalização dos labels
target = encoder.fit_transform(labels)

In [0]:
# Items
encoder.classes_.item(0)

In [0]:
# Fit do modelo
modelo.fit(data, target)

In [0]:
# Prevendo a categoria a partir da descrição
modelo.predict(["intel"])

In [0]:
# Prevendo a categoria a partir da descrição
print (encoder.classes_[11])

In [0]:
# Probabilidades de um produto
probs = modelo.predict_proba(["INTEL"])

In [0]:
# Probabidades de categorias para o objeto Ventilador
guess = [(class_, probs.item(n)) for n, class_ in enumerate(encoder.classes_)]
pprint(guess)

In [0]:
# Probabidade ajustada de categorias para o objeto iNTEL
from operator import itemgetter
for cat, proba in sorted(guess, key = itemgetter(1), reverse = True):
    print ('{}: {:.4f}'.format(cat, proba))

### Fim

### Comunidade Telegram - <a href=https://t.me/cienciadosdados>https://t.me/cienciadosdados</a>

### Ciência dos Dados - <a href=http://facebook.com/cienciadosdadosbr>facebook.com/cienciadosdadosbr</a>